# Pengambilan Data

In [25]:
topik = 'cryptocurrency'
waktu_ambil = '13 maret malam'

In [14]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [15]:
import twitter_credentials
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import string
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

In [17]:
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

In [18]:
# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)

In [19]:
# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
      
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                       tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)

In [20]:
 class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
         
    def clean_tweet(self, tweet):
    
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w +:\/\/ \S +)", " ", tweet).split())
    
    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        # analysis = analysis.translate(from_lang = 'in',to='en')

        if analysis.sentiment.polarity > 0:
            return "Positif"
        elif analysis.sentiment.polarity == 0:
            return "Netral"
        else:
            return "Negatif"
        
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])
        
        df['id'] = np.array([tweet.id_str for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['lang'] = np.array([tweet.lang for tweet in tweets])
        df['place'] = np.array([tweet.place for tweet in tweets ])
        

        return df

In [21]:
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()
    keyword = [topik]
    api = twitter_client.get_twitter_client_api()
    tweets = api.search(keyword, count=100, lang = 'en')

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    print(df)

                                               tweets                   id  \
0   RT @cryptanalyser: #BTCUSD Market #1H timefram...  1238468783208988673   
1   Bitcoin isn't magic. It behaves the way its ow...  1238468781866995712   
2   RT @APompliano: Treasury Secretary Mnuchin jus...  1238468781741166593   
3   @PeterSchiff $5,707 at block: 621,485.\n\nYou ...  1238468775227219968   
4   @Ben__Rickert @NickSzabo4 Something like this....  1238468768818495488   
5   As soon as I opened my eyes today after waking...  1238468763965624320   
6   @missing_clue @LibertyImplac I put in 100. If ...  1238468758873628675   
7   @hodl_american As long as he helps bitcoin cor...  1238468758819287040   
8   RT @Bitsahara: HODL #Bitcoin  !\n\nRT for 15,0...  1238468756352819200   
9   RT @iamjosephyoung: My prediction:\n\nBitMEX w...  1238468751944712196   
10  RT @Bex5002: 🎊#Bex500 will launch a trading re...  1238468745653256192   
11  Fast Bitcoin Pay By 60 Minutes! \nhttps://t.co...  123846874

# Preproses

In [22]:
def remove_punct(text):
  
    #hapus simbol
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    text = re.sub('rt @[^\s]+',' ', text)
    text = re.sub('@[^\s]+',' ',text)
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    #Remove additional white spaces
    text = re.sub('[\s]+', ' ', text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    #trim
    text = text.strip('\'"')
    text  = "".join([char for char in text if char not in string.punctuation])
    #text = re.sub('[0-9]+', '', text)

    return text

df['tweet_bersih'] = df['tweets'].apply(lambda x: remove_punct(x.lower()))

In [23]:
#--- REMOVE DUPLICATES
df = df.drop_duplicates(['tweet_bersih'])
df

,tweets,id,date,source,lang,place,sentiment,tweet_bersih
0,RT @cryptanalyser: #BTCUSD Market #1H timefram...,1238468783208988673,2020-03-13 14:15:42,Twitter Web App,en,None,Netral,btcusd market 1h timeframe on march 13 at 130...
1,Bitcoin isn't magic. It behaves the way its ow...,1238468781866995712,2020-03-13 14:15:41,Twitter Web App,en,None,Positif,bitcoin isnt magic it behaves the way its owne...
2,RT @APompliano: Treasury Secretary Mnuchin jus...,1238468781741166593,2020-03-13 14:15:41,Twitter for Android,en,None,Netral,treasury secretary mnuchin just said the fede...
3,"@PeterSchiff $5,707 at block: 621,485.\n\nYou ...",1238468775227219968,2020-03-13 14:15:40,Twitter for Android,en,None,Negatif,5707 at block 621485 you can still send bitco...
4,@Ben__Rickert @NickSzabo4 Something like this....,1238468768818495488,2020-03-13 14:15:38,Twitter for iPhone,en,None,Netral,something like this bitcoin
5,As soon as I opened my eyes today after waking...,1238468763965624320,2020-03-13 14:15:37,Twitter Web App,en,None,Netral,as soon as i opened my eyes today after waking...
6,@missing_clue @LibertyImplac I put in 100. If ...,1238468758873628675,2020-03-13 14:15:36,Twitter for Android,en,None,Netral,i put in 100 if i had waited two days i could...
7,@hodl_american As long as he helps bitcoin cor...,1238468758819287040,2020-03-13 14:15:36,Twitter for iPhone,en,None,Positif,as long as he helps bitcoin core he can belie...
8,"RT @Bitsahara: HODL #Bitcoin !\n\nRT for 15,0...",1238468756352819200,2020-03-13 14:15:35,Twitter for Android,en,None,Netral,hodl bitcoin rt for 15000 sats on lightning ...
9,RT @iamjosephyoung: My prediction:\n\nBitMEX w...,1238468751944712196,2020-03-13 14:15:34,Twitter for iPhone,en,None,Netral,my prediction bitmex wont be the dominant bit...


In [24]:
df.to_excel("Data Tanggal " + waktu_ambil + "(" + topik + ").xlsx")
df.to_csv("Data Tanggal " + waktu_ambil + "(" + topik + ").csv")